In [ ]:
# Install required packages if needed
import subprocess
import sys

try:
    import spacy_transformers
    print("spacy-transformers is already installed")
except ImportError:
    print("Installing spacy-transformers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "spacy-transformers"])

# Also ensure we have the latest transformers version
try:
    from transformers import __version__ as transformers_version
    print(f"transformers version: {transformers_version}")
except ImportError:
    print("Installing transformers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "transformers"])

In [5]:
# Import all required libraries
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import AutoModel
import torch
import spacy
import spacy_transformers  # Explicitly import spacy_transformers
from pathlib import Path
import subprocess
import os

print("All imports successful!")

All imports successful!


In [4]:
path=Path(r"model-best")
# Load the best trained model
nlp = spacy.load(path)

In [26]:
# Use a proper NER model instead of base BERT
model_name = "manishiitg/resume-ner"  # This is trained for NER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Available labels: {list(model.config.id2label.values())}")

Model loaded: manishiitg/resume-ner
Model max length: 512
Available labels: ['CARDINAL', 'GPE', 'Phone', 'Skills', 'ORG', 'EducationDegree', 'ExperianceYears', 'DATE', 'PERSON', 'Designation', 'O']


In [15]:
text="Ram Edupuganti - Resume Current: Software Development Director - Oracle Inc  - Email me on Indeed: indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21  \u2022 Offering over 22 years of rich national & international experience in developing & deploying software appplicatoins development, customized solutions across Oracle Fusion HCM Global Core HR, Global Payroll, Global Absences, Compensation and all Talent modules - Performance Management, Goals Management, Career Development, Succession Management & Talent Review Meeting \u2022 Directed planning, strategy development and implementation & business solution delivery with recent successful implantations at Schneider Electric, Macy's, Pike International, British Telcom. \u2022 Accomplished in relational data modeling, data warehouse design and implementation, object oriented design and development, developing transactional & business analytics in CRM, SCM and HCM domains from client server to cloud SaaS. \u2022 Led large-scale business application architecture and design efforts; developed and maintained solutions for various business functional areas; assisted in resolving integration and interface issues between various systems with focus on optimizing performance and scalability \u2022 Extensive experience in all aspects of project management including budgeting and cost optimization, risk assessments and control, technical feasibility studies, project scope definition, estimations & cost control & so on \u2022 Recruit, Develop and led high-performing teams with high motivation and enhanced cross- functional collaboration  WORK EXPERIENCE  Software Development Director  Oracle Inc -  April 2015 to Present  Employer: Oracle Inc -  January 1998 to Present  98 with Oracle Inc. Growth Path & Deputations:  Role: Senior Development Manager  Employer: Oracle Inc -  December 2011 to March 2015  Senior Development Manager  Employer: Oracle Inc -  March 2010 to November 2011  https://www.indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21?isid=rex-download&ikw=download-top&co=IN   Role: Principal Applications Developer  Oracle Inc -  September 2004 to February 2010  Project Lead  Oracle Inc -  September 1999 to August 2004  Senior Software Developer  Oracle Inc, Inc. -  January 1998 to August 1999  R-Systems -  Sacramento, CA -  January 1997 to January 1998  Key Result Areas: \u2022 Providing overall leadership to the entire project team, mapping clients' requirements, transforming requirements into stipulations and providing them best solutions within the scope of project requirements \u2022 Creating and managing the estimates, project plan, project schedule, resource allocation and expenses; monitoring and reporting on standards & performance targets \u2022 Ensuring the delivery of quality releases on schedules using continuous improvement initiatives; adhering to quality norms throughout the project implementation \u2022 Working on service strategy, transition, operations, process improvement, process management, team building, training, hiring and client relationship management \u2022 Identifying and implementing strategies for building team effectiveness by promoting a spirit of cooperation among the team members \u2022 Supporting continuous improvement by investigating alternatives and new technologies and presenting the same for architectural review \u2022 Liaising with stakeholders during the course of problem diagnoses, requirements gathering, detailed level design, development, system test and production implementation to ensure that optimal resolutions are achieved  KNOWLEDGE PURVIEW \u2022 Machine Learning Algorithms supervised, unsupervised, NLP, chat bot and deep learning with Python \u2022 Building transactional applications & analytical solutions in SaaS model \u2022 Oracle fusion role based data security set up and customization for complex security requirements \u2022 Reporting Layer (Subject Areas) for fusion HCM cloud for all HCM modules \u2022 Data Warehousing design methodologies, star and snowflake schema designs, aggregations \u2022 OBIEE, Data Visualization Desktop, BICS, DVCS, Oracle Analytics Cloud (OAC), Tableau \u2022 Design and build dashboards, KPIs using Oracle BI platform and OTBI, Standard Reports using BI Publisher \u2022 Design and build mappings, knowledge modules, load plans using Oracle Data Integrator (ODI)    \u2022 Relational data modeling, Object Oriented Modeling & Design, UML, SOAP \u2022 Oracle 10g, 11g, 12c \u2022 SQL, PL/SQL, HTML, XML, JAVA, JDK, J2EE, Oracle ADF, Oracle JDeveloper \u2022 Applications Development for Oracle CRM, Oracle ebusiness SCM, \u2022 Applications/database/SQL/Batch program performance tuning \u2022 UNIX, Windows, Linux  R-Systems -  April 1996 to January 1998  Growth Path & Deputations:  Department of Corrections -  Sacramento, CA -  February 1996 to January 1997  Secretary of State  Salem, Tamil Nadu -  December 1995 to February 1996  Oracle Corporation -  Redwood Shores, CA -  January 1995 to December 1995  EDUCATION  M.S. in Computer Engineering with honors  Texas A&M University -  Kingsville, TX  1994  B.S. in Electronics & Communications Engineering with honors  Nagarjuna University  1992  SKILLS  SOFTWARE DEVELOPMENT (3 years), STRUCTURED SOFTWARE (3 years), BUSINESS INTELLIGENCE (1 year), ORACLE (1 year), ARCHITECTURE (Less than 1 year)  ADDITIONAL INFORMATION  CORE COMPETENCIES  Software Development & Consulting  Oracle Fusion Applications Architecture and Functionality    Oracle Business Intelligence development and implementations  Technology Planning  Delivery Management  Client Engagements (Stakeholders/Business)  Continuous Process Enhancement  Agile/Scrum Methodologies  Project Management"


In [ ]:
inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    is_split_into_words=False
)

# 2. Forward pass
with torch.no_grad():
    outputs = model(**inputs)          # outputs.logits shape: [batch, seq_len, num_labels]

# 3. Take the most probable label for each token
pred_ids = torch.argmax(outputs.logits, dim=2)[0]    # shape: [seq_len]

# 4. Convert back to words + labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
labels = [model.config.id2label[id_.item()] for id_ in pred_ids]



j=-1
for i, (token, label) in enumerate(zip(tokens, labels)):
    if(j>i):
        continue
    if label != "O":
        # Start a span with the current token
        span = token
        j = i + 1
        while j < len(labels) and labels[j] == label:# Check consecutive tokens with the same label
            span +=" "
            span += tokenizer.convert_tokens_to_string([tokens[j]])# Concatenate subsequent tokens with the same label
            j += 1
        print(f"{span:<30} {label}")

software development director  Designation
oracle inc                     ORG
22 years                       ExperianceYears
april 2015                     DATE
present                        DATE
january 1998                   DATE
present 98                     DATE
senior development manager     Designation
december 2011                  DATE
march 2015                     DATE
senior development manager     Designation
march 2010                     DATE
november 2011                  DATE
applications                   Designation
september 2004                 DATE
february 2010                  DATE
september 1999                 DATE
august 2004                    DATE
senior software developer      Designation
january 1998                   DATE
august 1999                    DATE
january 1997                   DATE
january 1998                   DATE
